In [13]:
from bs4 import BeautifulSoup
import csv

# Define base file name
base_file_name = "C:/Users/santi/Desktop/TCG/TCGplayer_ Shop YuGiOh Cards, Packs, Booster Boxes"

# Create and open the CSV file
with open('card_data.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['Card Name', 'Rarity', 'Number', 'Price', 'Market Price', 'Listing Count'])

    # Iterate over each file
    for i in range(1, 41):  # Adjust the range based on how many files you have
        file_name = base_file_name + str(i) + ".html"

        # Open the HTML file
        with open(file_name, 'r', encoding='utf-8') as html_file:
            html = html_file.read()

        # Create a BeautifulSoup object
        soup = BeautifulSoup(html, 'html.parser')

        # Find all 'section' tags with class 'search-result__product'
        card_sections = soup.find_all('section', {'class': 'search-result__product'})

        # Iterate over each card section
        for card in card_sections:

            # Extract the card name
            card_name_tag = card.find('span', {'class': 'search-result__title'})
            card_name = card_name_tag.text if card_name_tag else ''

            # Extract the rarity and number
            rarity_section = card.find('section', {'class': 'search-result__rarity'})
            if rarity_section:
                rarity_spans = rarity_section.find_all('span')
                if len(rarity_spans) >= 3:
                    rarity = rarity_spans[0].text
                    number = rarity_spans[2].text
            else:
                rarity = ''
                number = ''

            # Extract the price
            price_tag = card.find('span', {'class': 'inventory__price-with-shipping'})
            price = price_tag.text if price_tag else ''

            listing_container = card.find('div', {'class': 'inventory__container'})
            if listing_container:
                listing_count_tag = listing_container.find('span', {'class': 'inventory__listing-count'})
                if listing_count_tag:
                    listing_count_text = listing_count_tag.text
                    listing_count = int(listing_count_text.split()[0])
                else:
                    listing_count = ''
            market_price = soup.find('span', class_='search-result__market-price--value').text
            # Write the card data to the CSV file
            writer.writerow([card_name, rarity, number, price, market_price, listing_count])


In [14]:
import praw
import pandas as pd
from collections import defaultdict

# Read your CSV file into a pandas DataFrame
df = pd.read_csv("C:/Users/santi/Desktop/VSCODEWORKING/card_data.csv")
card_list = df['Card Name'].tolist()

# Prepare to count mentions
mentions = defaultdict(int, {card: 0 for card in card_list})


# Create a Reddit instance
reddit = praw.Reddit(
    client_id='Cmr3W3e6ewCr728I1DgZ9Q',
    client_secret='jxbtT-nxuvriUNJTOa2WEij0GZM1Eg',
    user_agent='yugioh card counter V1',
    username='sneeze820',
    password='Santi8/21/06',
)

# Set your subreddit
subreddit = reddit.subreddit('yugioh')

# Loop through each post in the subreddit
for post in subreddit.new(limit=1000):
    # Check each card in the card list
    for card in card_list:
        # If the card name is in the post title or content, increment the count
        if card.lower() in post.title.lower() or card.lower() in post.selftext.lower():
            mentions[card] += 1

# Print the results
# Add a new column to the DataFrame for the mention counts
df['Mention Count'] = df['Card Name'].map(mentions)

# Write the DataFrame back to the CSV file
df.to_csv('C:/Users/santi/Desktop/VSCODEWORKING/card_data.csv', index=False)


In [13]:
new_directory = "C:/Users/santi/Desktop/VSCODEWORKING"

# Change the working directory
os.chdir(new_directory)

# Check the new working directory
print("Current Working Directory:", os.getcwd())

Current Working Directory: C:\Users\santi\Desktop\VSCODEWORKING
